# Unity Catalog Governance

**KION Training - Dzień 3**

---

## 📚 Agenda

1. Unity Catalog Architecture
2. Zarządzanie dostępami: GRANT / REVOKE
3. Data Masking i Row-Level Security
4. Data Lineage i Audit Logging
5. Delta Sharing
6. Best Practices for Data Governance

---

## 🎯 Cele szkolenia

Po tym module będziesz potrafić:
- Rozumieć architekturę Unity Catalog
- Zarządzać uprawnieniami i rolami
- Implementować data masking i row-level security
- Śledzić data lineage
- Konfigurować Delta Sharing

---

## 1️⃣ Unity Catalog Architecture

**Unity Catalog** to zunifikowane rozwiązanie governance dla Databricks Lakehouse.

### Hierarchia obiektów:

```
Metastore (region-level)
    ↓
Catalog (database/domain)
    ↓
Schema (namespace)
    ↓
Securable Objects:
    - Tables / Views
    - Functions (UDF, stored procedures)
    - Volumes (files storage)
    - Models (ML models)
```

### Three-level namespace:
```sql
catalog.schema.table
```

Przykład:
```sql
main.sales.orders
dev.analytics.customer_metrics
prod.gold.daily_revenue
```

### Kluczowe cechy:
- **Unified governance**: jedna platforma dla danych, ML, BI
- **Fine-grained access control**: table, column, row level
- **Automatic lineage**: end-to-end data flow tracking
- **Audit logging**: who accessed what and when
- **Data discovery**: metadata search i tagging

---

## 📋 Setup i Basic Operations

### Creating Catalogs and Schemas:

In [ ]:
# Create Catalog
spark.sql("""
    CREATE CATALOG IF NOT EXISTS kion_prod
    COMMENT 'KION Production catalog'
""")

spark.sql("""
    CREATE CATALOG IF NOT EXISTS kion_dev
    COMMENT 'KION Development catalog'
""")

# List catalogs
spark.sql("SHOW CATALOGS").display()

In [ ]:
# Create Schemas within catalog
spark.sql("""
    CREATE SCHEMA IF NOT EXISTS kion_prod.bronze
    COMMENT 'Bronze layer - raw data'
""")

spark.sql("""
    CREATE SCHEMA IF NOT EXISTS kion_prod.silver
    COMMENT 'Silver layer - cleaned data'
""")

spark.sql("""
    CREATE SCHEMA IF NOT EXISTS kion_prod.gold
    COMMENT 'Gold layer - business aggregations'
""")

# List schemas
spark.sql("SHOW SCHEMAS IN kion_prod").display()

In [ ]:
# Set default catalog and schema
spark.sql("USE CATALOG kion_prod")
spark.sql("USE SCHEMA silver")

# Now you can reference tables without full path:
# SELECT * FROM orders
# Instead of: SELECT * FROM kion_prod.silver.orders

### Creating Tables in Unity Catalog:

In [ ]:
from pyspark.sql.functions import *

# Load sample data
orders_df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/main/default/kion_data/orders/*.csv")
)

# Write to Unity Catalog managed table
orders_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("kion_prod.silver.orders")

print("✅ Table created: kion_prod.silver.orders")

In [ ]:
# Add table properties and comments
spark.sql("""
    ALTER TABLE kion_prod.silver.orders
    SET TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true',
        'owner' = 'data-engineering-team',
        'department' = 'analytics',
        'pii_data' = 'true'
    )
""")

spark.sql("""
    COMMENT ON TABLE kion_prod.silver.orders IS
    'Cleaned orders table with data quality validations applied'
""")

spark.sql("""
    COMMENT ON COLUMN kion_prod.silver.orders.customer_id IS
    'Customer identifier - PII data, access restricted'
""")

### Inspecting metadata:

In [ ]:
# Describe table
spark.sql("DESCRIBE EXTENDED kion_prod.silver.orders").display()

In [ ]:
# Show table properties
spark.sql("SHOW TBLPROPERTIES kion_prod.silver.orders").display()

In [ ]:
# List all tables in schema
spark.sql("SHOW TABLES IN kion_prod.silver").display()

---

## 2️⃣ Zarządzanie dostępami: GRANT / REVOKE

### Privilege hierarchy w Unity Catalog:

```
Metastore privileges:
    - CREATE CATALOG
    - USE CATALOG

Catalog privileges:
    - USE CATALOG
    - CREATE SCHEMA
    - USE SCHEMA

Schema privileges:
    - USE SCHEMA
    - CREATE TABLE
    - CREATE FUNCTION
    - CREATE VOLUME

Table privileges:
    - SELECT
    - MODIFY (INSERT, UPDATE, DELETE, MERGE)
    - READ_METADATA
```

### Basic GRANT/REVOKE patterns:

In [ ]:
# Grant catalog access to data analysts
spark.sql("""
    GRANT USE CATALOG ON CATALOG kion_prod TO `data-analysts`
""")

spark.sql("""
    GRANT USE SCHEMA ON SCHEMA kion_prod.gold TO `data-analysts`
""")

# Analysts can read gold tables
spark.sql("""
    GRANT SELECT ON SCHEMA kion_prod.gold TO `data-analysts`
""")

print("✅ Granted access to data-analysts group")

In [ ]:
# Grant full access to data engineers
spark.sql("""
    GRANT USE CATALOG, CREATE SCHEMA ON CATALOG kion_prod TO `data-engineers`
""")

spark.sql("""
    GRANT ALL PRIVILEGES ON SCHEMA kion_prod.bronze TO `data-engineers`
""")

spark.sql("""
    GRANT ALL PRIVILEGES ON SCHEMA kion_prod.silver TO `data-engineers`
""")

spark.sql("""
    GRANT ALL PRIVILEGES ON SCHEMA kion_prod.gold TO `data-engineers`
""")

print("✅ Granted full access to data-engineers group")

In [ ]:
# Grant specific table access
spark.sql("""
    GRANT SELECT ON TABLE kion_prod.gold.daily_sales TO `finance-team`
""")

spark.sql("""
    GRANT SELECT ON TABLE kion_prod.gold.customer_metrics TO `marketing-team`
""")

print("✅ Granted table-specific access")

In [ ]:
# Revoke access
spark.sql("""
    REVOKE SELECT ON TABLE kion_prod.silver.orders FROM `contractors`
""")

print("✅ Revoked access from contractors group")

In [ ]:
# Show grants on object
spark.sql("""
    SHOW GRANTS ON TABLE kion_prod.silver.orders
""").display()

In [ ]:
# Show grants for principal (user/group)
spark.sql("""
    SHOW GRANTS ON CATALOG kion_prod
""").display()

### Ownership and transfer:

In [ ]:
# Change owner of catalog
spark.sql("""
    ALTER CATALOG kion_prod OWNER TO `data-engineering-team`
""")

# Change owner of schema
spark.sql("""
    ALTER SCHEMA kion_prod.gold OWNER TO `analytics-team`
""")

# Change owner of table
spark.sql("""
    ALTER TABLE kion_prod.gold.daily_sales OWNER TO `finance-team`
""")

print("✅ Ownership transferred")

---

## 3️⃣ Data Masking i Row-Level Security

### Column-level masking (Dynamic Views):

Użyj funkcji `current_user()` i `is_account_group_member()` do conditional masking:

In [ ]:
# Create masked view for PII data
spark.sql("""
    CREATE OR REPLACE VIEW kion_prod.gold.customers_masked AS
    SELECT 
        customer_id,
        CASE 
            WHEN is_account_group_member('pii-access-group') THEN email
            ELSE '***@*****.***'
        END as email,
        CASE 
            WHEN is_account_group_member('pii-access-group') THEN phone
            ELSE 'XXX-XXX-XXXX'
        END as phone,
        CASE 
            WHEN is_account_group_member('pii-access-group') THEN full_name
            ELSE CONCAT(LEFT(first_name, 1), '*** ', LEFT(last_name, 1), '***')
        END as full_name,
        city,
        country,
        lifetime_value,
        registration_date
    FROM kion_prod.silver.customers
""")

print("✅ Created masked view for customers table")

In [ ]:
# Test masking
# Users IN 'pii-access-group' see real data
# Other users see masked data
spark.sql("""
    SELECT * FROM kion_prod.gold.customers_masked LIMIT 10
""").display()

In [ ]:
# Alternative: Hash sensitive data
spark.sql("""
    CREATE OR REPLACE VIEW kion_prod.gold.orders_hashed AS
    SELECT 
        order_id,
        SHA2(CAST(customer_id AS STRING), 256) as customer_id_hash,
        order_date,
        amount,
        status
    FROM kion_prod.silver.orders
""")

# Analysts can work with hashed IDs for aggregations
# Without seeing actual customer identifiers

### Row-Level Security (RLS):

Restrict which rows users can see based on their identity or group membership:

In [ ]:
# Create row-level security view - regional access
spark.sql("""
    CREATE OR REPLACE VIEW kion_prod.gold.orders_rls AS
    SELECT *
    FROM kion_prod.silver.orders
    WHERE 
        CASE 
            WHEN is_account_group_member('global-access') THEN TRUE
            WHEN is_account_group_member('europe-team') THEN region = 'EMEA'
            WHEN is_account_group_member('americas-team') THEN region = 'Americas'
            WHEN is_account_group_member('asia-team') THEN region = 'APAC'
            ELSE FALSE
        END
""")

print("✅ Row-level security applied: users see only their region's data")

In [ ]:
# RLS based on user attribute (e.g., department)
spark.sql("""
    CREATE OR REPLACE VIEW kion_prod.gold.sales_rls AS
    SELECT 
        o.*,
        d.department
    FROM kion_prod.silver.orders o
    JOIN kion_prod.silver.departments d ON o.department_id = d.department_id
    WHERE 
        is_account_group_member('admin') OR
        current_user() IN (
            SELECT user_email 
            FROM kion_prod.gold.user_department_mapping 
            WHERE department = d.department
        )
""")

# Users only see sales from their own department

In [ ]:
# Grant access to RLS view
spark.sql("""
    GRANT SELECT ON VIEW kion_prod.gold.orders_rls TO `all-users`
""")

# Revoke direct access to base table
spark.sql("""
    REVOKE SELECT ON TABLE kion_prod.silver.orders FROM `all-users`
""")

print("✅ Users can only access data through RLS view")

---

## 4️⃣ Data Lineage i Audit Logging

### Querying Data Lineage:

Unity Catalog automatically tracks lineage for:
- Table → Table (ETL transformations)
- Notebook → Table (data writes)
- Dashboard → Table (BI queries)
- ML Model → Table (training data)

In [ ]:
# Query table lineage from system tables
lineage_df = spark.sql("""
    SELECT 
        source_table_full_name,
        source_type,
        target_table_full_name,
        target_type,
        created_at,
        created_by
    FROM system.access.table_lineage
    WHERE target_table_full_name LIKE 'kion_prod.%'
    ORDER BY created_at DESC
    LIMIT 50
""")
lineage_df.display()

In [ ]:
# Find upstream dependencies (sources) for a table
upstream_df = spark.sql("""
    SELECT DISTINCT
        source_table_full_name,
        source_type
    FROM system.access.table_lineage
    WHERE target_table_full_name = 'kion_prod.gold.daily_sales'
""")
upstream_df.display()

print("⬆️ Upstream tables that feed into gold.daily_sales")

In [ ]:
# Find downstream dependencies (consumers) of a table
downstream_df = spark.sql("""
    SELECT DISTINCT
        target_table_full_name,
        target_type
    FROM system.access.table_lineage
    WHERE source_table_full_name = 'kion_prod.silver.orders'
""")
downstream_df.display()

print("⬇️ Downstream tables that consume silver.orders")

In [ ]:
# Column-level lineage (if available)
column_lineage = spark.sql("""
    SELECT 
        source_table_full_name,
        source_column_name,
        target_table_full_name,
        target_column_name,
        created_at
    FROM system.access.column_lineage
    WHERE target_table_full_name = 'kion_prod.gold.customer_metrics'
    ORDER BY target_column_name
""")
column_lineage.display()

print("📊 Column-level lineage for customer_metrics")

### Audit Logging:

Unity Catalog logs all access and operations:

In [ ]:
# Query audit logs
audit_df = spark.sql("""
    SELECT 
        event_time,
        user_identity.email as user_email,
        service_name,
        action_name,
        request_params.full_name_arg as table_name,
        response.status_code,
        request_id
    FROM system.access.audit
    WHERE action_name IN ('getTable', 'createTable', 'deleteTable', 'updateTable')
        AND event_date >= current_date() - INTERVAL 7 DAYS
    ORDER BY event_time DESC
    LIMIT 100
""")
audit_df.display()

In [ ]:
# Track who accessed sensitive tables
sensitive_access = spark.sql("""
    SELECT 
        event_time,
        user_identity.email as user,
        action_name,
        request_params.full_name_arg as table_accessed,
        source_ip_address
    FROM system.access.audit
    WHERE request_params.full_name_arg LIKE '%customers%'
        AND action_name = 'getTable'
        AND event_date >= current_date() - INTERVAL 1 DAYS
    ORDER BY event_time DESC
""")
sensitive_access.display()

print("🔒 Access logs for sensitive customer data")

In [ ]:
# Grant/Revoke audit trail
grant_audit = spark.sql("""
    SELECT 
        event_time,
        user_identity.email as admin_user,
        action_name,
        request_params.privilege as privilege_granted,
        request_params.securable_full_name as object_name,
        request_params.principal as grantee
    FROM system.access.audit
    WHERE action_name IN ('grantPrivilege', 'revokePrivilege')
        AND event_date >= current_date() - INTERVAL 30 DAYS
    ORDER BY event_time DESC
""")
grant_audit.display()

print("📝 Audit trail of privilege changes")

---

## 5️⃣ Delta Sharing

**Delta Sharing** = Secure data sharing protocol (cross-org, cross-cloud)

### Komponenty:
- **Share**: kolekcja tabel do udostępnienia
- **Recipient**: organizacja/użytkownik otrzymujący dane
- **Provider**: właściciel danych (Ty)

### Create Share:

In [ ]:
# Create a share
spark.sql("""
    CREATE SHARE IF NOT EXISTS kion_partner_share
    COMMENT 'Shared data for KION business partners'
""")

print("✅ Share created: kion_partner_share")

In [ ]:
# Add tables to share
spark.sql("""
    ALTER SHARE kion_partner_share
    ADD TABLE kion_prod.gold.daily_sales
""")

spark.sql("""
    ALTER SHARE kion_partner_share
    ADD TABLE kion_prod.gold.product_catalog
""")

print("✅ Tables added to share")

In [ ]:
# List shares
spark.sql("SHOW SHARES").display()

In [ ]:
# Show what's in a share
spark.sql("SHOW ALL IN SHARE kion_partner_share").display()

### Create Recipient:

In [ ]:
# Create recipient (partner organization)
spark.sql("""
    CREATE RECIPIENT IF NOT EXISTS partner_acme_corp
    COMMENT 'ACME Corp partner access'
""")

print("✅ Recipient created: partner_acme_corp")

In [ ]:
# Grant share to recipient
spark.sql("""
    GRANT SELECT ON SHARE kion_partner_share TO RECIPIENT partner_acme_corp
""")

print("✅ Share granted to recipient")

In [ ]:
# List recipients
spark.sql("SHOW RECIPIENTS").display()

In [ ]:
# Describe recipient (get activation link)
spark.sql("DESCRIBE RECIPIENT partner_acme_corp").display()

# Recipient uses activation link to access shared data

### Consuming shared data (as recipient):

In [ ]:
# As a recipient, you create a catalog from the share
# spark.sql("""
#     CREATE CATALOG partner_data
#     USING SHARE provider.kion_partner_share
# """)

# Then query the shared tables:
# SELECT * FROM partner_data.gold.daily_sales

# Shared data is read-only - no modifications allowed

### Best practices for Delta Sharing:

1. **Share only aggregated/gold data**: nie udostępniaj raw/bronze layers
2. **Use views for masking**: create view with masked PII before sharing
3. **Monitor access**: track who accesses shared data
4. **Version control**: use table versions for stable APIs
5. **Documentation**: clear documentation dla recipients

---

## 6️⃣ Best Practices for Data Governance

### 1. Catalog organization strategy:

In [ ]:
# Recommended catalog structure:

# Environment-based:
# - dev (development)
# - test (testing/QA)
# - prod (production)

# Domain-based within catalog:
# - sales_domain
# - marketing_domain
# - finance_domain

# Layer-based schemas:
# - bronze (raw)
# - silver (cleaned)
# - gold (aggregated)

# Example:
# prod.sales_domain.bronze_orders
# prod.sales_domain.silver_orders
# prod.sales_domain.gold_daily_sales

### 2. Access control patterns:

In [ ]:
# Principle of Least Privilege:
# - Data Engineers: Full access to bronze/silver, SELECT on gold
# - Data Scientists: SELECT on silver/gold
# - Analysts: SELECT on gold only
# - Executives: SELECT on gold dashboards only

# Use groups, not individual users:
# GRANT ... TO `data-engineers` -- ✅
# GRANT ... TO `john.doe@company.com` -- ❌

### 3. Tagging and documentation:

In [ ]:
# Use table properties for metadata
spark.sql("""
    ALTER TABLE kion_prod.silver.orders
    SET TBLPROPERTIES (
        'data_classification' = 'confidential',
        'pii_fields' = 'customer_id,email,phone',
        'retention_days' = '2555',  -- 7 years
        'owner_team' = 'data-engineering',
        'data_domain' = 'sales',
        'refresh_frequency' = 'daily',
        'sla_hours' = '24'
    )
""")

# Comment every table and important columns
spark.sql("""
    COMMENT ON TABLE kion_prod.silver.orders IS
    'Customer orders with validation and cleaning applied. 
     Updated daily at 2 AM. Contains PII data.'
""")

### 4. Monitoring and alerts:

In [ ]:
# Regular governance health checks

# 1. Tables without owners
unowned_tables = spark.sql("""
    SELECT 
        table_catalog,
        table_schema,
        table_name
    FROM system.information_schema.tables
    WHERE table_catalog = 'kion_prod'
        AND table_owner IS NULL
""")
unowned_tables.display()

# 2. Tables without comments
undocumented = spark.sql("""
    SELECT 
        table_catalog,
        table_schema,
        table_name
    FROM system.information_schema.tables
    WHERE table_catalog = 'kion_prod'
        AND (comment IS NULL OR comment = '')
""")
undocumented.display()

# 3. Unused tables (no queries in 90 days)
unused_tables = spark.sql("""
    WITH recent_access AS (
        SELECT DISTINCT request_params.full_name_arg as table_name
        FROM system.access.audit
        WHERE action_name = 'getTable'
            AND event_date >= current_date() - INTERVAL 90 DAYS
    )
    SELECT 
        t.table_catalog,
        t.table_schema,
        t.table_name,
        t.created as table_created_at
    FROM system.information_schema.tables t
    LEFT JOIN recent_access ra 
        ON CONCAT(t.table_catalog, '.', t.table_schema, '.', t.table_name) = ra.table_name
    WHERE t.table_catalog = 'kion_prod'
        AND ra.table_name IS NULL
        AND t.created < current_date() - INTERVAL 90 DAYS
""")
unused_tables.display()

---

## ✅ Podsumowanie

### Nauczyłeś się:

✅ **Unity Catalog Architecture**: Metastore → Catalog → Schema → Tables  
✅ **Access Control**: GRANT/REVOKE privileges at multiple levels  
✅ **Data Masking**: Column-level masking with dynamic views  
✅ **Row-Level Security**: Filter data based on user identity  
✅ **Data Lineage**: Track data flow through system tables  
✅ **Audit Logging**: Monitor who accessed what and when  
✅ **Delta Sharing**: Secure cross-organization data sharing  

### Key Takeaways:

1. **Unified Governance**: Single platform for all data assets
2. **Fine-grained Control**: Table, column, row-level security
3. **Automatic Lineage**: No extra instrumentation needed
4. **Compliance-ready**: Audit logs for regulatory requirements
5. **Secure Sharing**: Delta Sharing for external collaboration

### Następne kroki:
- **Notebook 05**: BI & ML Integrations
- **Workshop 03**: Governance + Integrations hands-on

---

## 📚 Dodatkowe zasoby

- [Unity Catalog Documentation](https://docs.databricks.com/data-governance/unity-catalog/index.html)
- [Delta Sharing Protocol](https://delta.io/sharing/)
- [Unity Catalog Best Practices](https://docs.databricks.com/data-governance/unity-catalog/best-practices.html)

---